## Go to this given URL and solve the following questions.

> URL: https://www.youtube.com/@PW-Foundation/videos

In [4]:
import requests
from bs4 import BeautifulSoup
from pandas import DataFrame
from json import loads


url = 'https://www.youtube.com/@PW-Foundation/videos'

# Get the html by get method
r = requests.get(url)
print(r)

# Create BeautifulSoup object
soup = BeautifulSoup(r.text, 'html.parser')
all_script_tags = soup.findAll('script')

<Response [200]>


So far, this code imports necessary modules for web scraping, sends an HTTP GET request to a YouTube channel URL, creates a BeautifulSoup object from the HTML code of the web page, and finds all the script tags in the web page.

The resulting list all_script_tags contains all the script tags in the web page, which can be used to extract the ytInitialData JSON object that contains data for the videos on the channel. We can use the script_tag_to_json() function and the get_contents_dict() function defined earlier to extract this data.

Next, we can process the data to extract specific information about each video, such as the title, views, likes, and comments. We can then use this data to create a Pandas DataFrame, which can be used for further analysis and visualization.

In [5]:
def script_tag_to_json(tags: list) -> dict:
    for tag in reversed(tags):
        text: str = tag.text
        if 'ytInitialData = {"responseContext"' in text:
            return loads(text[20:-1])

    raise ValueError('Required script tag not found in the given tags.')

This code defines a function script_tag_to_json() that converts a script tag to a JSON object.

The function takes a list of script tags as input and searches for the tag that contains the JSON data that we are interested in. It does this by iterating over the list of tags in reverse order (starting from the end of the list), and searching for the ytInitialData JSON object in the text attribute of each tag.

If the required ytInitialData JSON object is found, the function extracts the JSON string from the text attribute of the tag (excluding the first 20 and last 1 characters, which contain some unwanted characters), and uses the loads() method from the json module to convert the string to a Python dictionary.

If the required ytInitialData JSON object is not found in any of the tags, the function raises a ValueError.

The resulting dictionary contains the data that we are interested in, which can be used for further analysis or manipulation.

In [6]:
data = script_tag_to_json(all_script_tags)

#Return data from videos
def get_contents_dict(data):
    return data['contents']['twoColumnBrowseResultsRenderer']['tabs'][1]['tabRenderer']['content']['richGridRenderer']['contents']

This code defines a function get_contents_dict() that extracts the contents dictionary from the ytInitialData JSON object.

The function takes the ytInitialData dictionary as input and returns the contents dictionary that contains the data for the videos on the channel.

The contents dictionary can be found by navigating through the various keys in the ytInitialData dictionary. In this case, we can find it by following this path:

In [ ]:
'contents'
    'twoColumnBrowseResultsRenderer'
        'tabs'
            [1]
                'tabRenderer'
                    'content'
                        'richGridRenderer'
                            'contents'


The resulting contents dictionary contains a list of dictionaries, where each dictionary represents a video on the channel. This data can be further processed and analyzed to extract specific information about each video, such as the title, views, likes, and comments.

### Q1. Write a python program to extract the video URL of the first five videos. 


#### Get Video ID

In [8]:
def get_videoUrl(data:dict, n: int = 5):
    contents = get_contents_dict(data)

    if n > 30:
        raise ValueError('Max Limit is 30.')

    result = []
    for i in range(n):
        result.append('https://www.youtube.com/watch?v=' +
                      contents[i]['richItemRenderer']['content']['videoRenderer']['videoId'])

    return result

get_videoUrl(data)

['https://www.youtube.com/watch?v=-osuIPJkPWg',
 'https://www.youtube.com/watch?v=VsKb_U2-Cx0',
 'https://www.youtube.com/watch?v=ZNHlCezYN1I',
 'https://www.youtube.com/watch?v=nNmV-fuGW5c',
 'https://www.youtube.com/watch?v=6Mht7UigC_w']

This code defines a function get_videoUrl() that extracts the URLs of the first n videos on the channel.

The function takes the ytInitialData dictionary as input and an optional parameter n that specifies the number of videos to retrieve (default value is 5). The function first extracts the contents dictionary using the get_contents_dict() function, and then iterates over the first n items in the contents list.

For each item, the function extracts the videoId from the videoRenderer dictionary and concatenates it with the YouTube video URL to create the full URL for the video. The resulting list result contains the URLs of the first n videos on the channel.

Note that the function raises a ValueError if the value of n is greater than 30, which is the maximum number of videos that can be retrieved from the YouTube API in a single request. This limit is enforced to prevent excessive usage of the API, which could result in rate limiting or other errors.

### Q2. Write a python program to extract the URL of the video thumbnails of the first five videos. 


#### Get video thumbnails

In [9]:
def get_thumbnails(data: dict, n: int = 5):
    contents = get_contents_dict(data)

    if n > 30:
        raise ValueError('Max Limit is 30.')

    result = []
    for i in range(n):
        result.append(contents[i]['richItemRenderer']['content']['videoRenderer']['thumbnail']['thumbnails'][-1]['url'])

    return result

get_thumbnails(data)

['https://i.ytimg.com/vi/-osuIPJkPWg/hqdefault.jpg?sqp=-oaymwEjCNACELwBSFryq4qpAxUIARUAAAAAGAElAADIQj0AgKJDeAE=&rs=AOn4CLAT-mWKPWrF_ucA3mmQN6nQwpioZg',
 'https://i.ytimg.com/vi/VsKb_U2-Cx0/hqdefault.jpg?sqp=-oaymwEjCNACELwBSFryq4qpAxUIARUAAAAAGAElAADIQj0AgKJDeAE=&rs=AOn4CLAMERoC6wPG1R1nxtOZR_4qRsKnNA',
 'https://i.ytimg.com/vi/ZNHlCezYN1I/hqdefault.jpg?sqp=-oaymwEjCNACELwBSFryq4qpAxUIARUAAAAAGAElAADIQj0AgKJDeAE=&rs=AOn4CLCeC-GzAbJasTeW24Z9mSnL8PUV0w',
 'https://i.ytimg.com/vi/nNmV-fuGW5c/hqdefault.jpg?sqp=-oaymwEjCNACELwBSFryq4qpAxUIARUAAAAAGAElAADIQj0AgKJDeAE=&rs=AOn4CLDrH1kaO1h12A7THP8j1zoeswW85w',
 'https://i.ytimg.com/vi/6Mht7UigC_w/hqdefault.jpg?sqp=-oaymwEjCNACELwBSFryq4qpAxUIARUAAAAAGAElAADIQj0AgKJDeAE=&rs=AOn4CLDT7e58gn2v84CHXh5IwWB6rXof8A']

This code defines a function get_thumbnails() that extracts the URLs of the thumbnail images for the first n videos on the channel.

The function takes the ytInitialData dictionary as input and an optional parameter n that specifies the number of thumbnails to retrieve (default value is 5). The function first extracts the contents dictionary using the get_contents_dict() function, and then iterates over the first n items in the contents list.

For each item, the function extracts the URL of the highest resolution thumbnail image by selecting the last item in the thumbnails list of the thumbnail dictionary.

The resulting list result contains the URLs of the thumbnail images for the first n videos on the channel. These URLs can be used to download and display the thumbnail images for each video.

### Q3. Write a python program to extract the title of the first five videos. 


#### Get video title

In [10]:
def get_title(data: dict, n:int = 5):
    contents = get_contents_dict(data)

    if n > 30:
        raise ValueError('Max Limit is 30.')

    result = []
    for i in range(n):
        result.append(contents[i]['richItemRenderer']['content']['videoRenderer']['title']['runs'][-1]['text'])

    return result

get_title(data)

['Revise through PYQs || Magnetic Effects of Electric Current - Class 10th Physics',
 'Revise through PYQs || Human Eye and the Colourful World - Class 10th Physics',
 'Revise through PYQs || Electricity - Class 10th Physics',
 'Revise through PYQs || Light - Class 10th Physics',
 'NEW Batches for Class 10 & 9 - Session 2023-24 || NEEV and UDAAN Batch Launch 🚀']

This code defines a function get_title() that extracts the titles of the first n videos on the channel.

The function takes the ytInitialData dictionary as input and an optional parameter n that specifies the number of titles to retrieve (default value is 5). The function first extracts the contents dictionary using the get_contents_dict() function, and then iterates over the first n items in the contents list.

For each item, the function extracts the title of the video by selecting the last item in the runs list of the title dictionary.

The resulting list result contains the titles of the first n videos on the channel. These titles can be used to identify each video and to display the titles in a list or table.

### Q4. Write a python program to extract the number of views of the first five videos. 


#### Get video viwes

In [11]:
def get_viwes(data: dict, n: int = 5):
    contents = get_contents_dict(data)

    if n > 30:
        raise ValueError('Max Limit is 30.')

    result = []
    for i in range(n):
        result.append(int(contents[i]['richItemRenderer']['content']['videoRenderer']['viewCountText']['simpleText']
                      [:-6].replace(',', '')))

    return result

get_viwes(data)

[1885, 1385, 12549, 36326, 38426]

This code defines a function get_views() that extracts the view counts of the first n videos on the channel.

The function takes the ytInitialData dictionary as input and an optional parameter n that specifies the number of view counts to retrieve (default value is 5). The function first extracts the contents dictionary using the get_contents_dict() function, and then iterates over the first n items in the contents list.

For each item, the function extracts the view count of the video by selecting the simpleText value of the viewCountText dictionary. The view count is then converted to an integer by removing the last six characters (which represent " views") and any commas.

The resulting list result contains the view counts of the first n videos on the channel. These view counts can be used to measure the popularity of each video and to display the view counts in a list or table.

### Q5. Write a python program to extract the time of posting of video for the first five videos.

#### Get time of posting of video

In [12]:
def get_time_of_posting(data: dict, n: int = 5):
    contents = get_contents_dict(data)

    if n > 30:
        raise ValueError('Max Limit is 30.')

    result = []
    for i in range(n):
        result.append(contents[i]['richItemRenderer']['content']['videoRenderer']['publishedTimeText']['simpleText'])

    return result

get_time_of_posting(data)

['3 hours ago', '4 hours ago', '9 hours ago', '13 hours ago', '16 hours ago']

This code defines a function get_time_of_posting() that extracts the publication dates of the first n videos on the channel.

The function takes the ytInitialData dictionary as input and an optional parameter n that specifies the number of publication dates to retrieve (default value is 5). The function first extracts the contents dictionary using the get_contents_dict() function, and then iterates over the first n items in the contents list.

For each item, the function extracts the publication date of the video by selecting the simpleText value of the publishedTimeText dictionary.

The resulting list result contains the publication dates of the first n videos on the channel. These dates can be used to determine when each video was published and to display the publication dates in a list or table.

# `Note:` Save all the data scraped in the above questions in a CSV file.


## Save data in `CSV` format.

In [13]:
def get_channel_video_details(data: dict, n: int):
    thumbnails = get_thumbnails(data, n)
    time_of_posting = get_time_of_posting(data, n)
    titles = get_title(data, n)
    video_urls = get_videoUrl(data, n)

    main_data = list(zip(video_urls, titles, thumbnails, time_of_posting))
    
    df = DataFrame.from_dict(main_data)
    df.rename(
        columns={
            0: 'video_urls',
            1: 'title',
            2: 'thumbnail_url',
            3: 'time_of_posting'
        }, inplace=True)

    return df

channel_data = get_channel_video_details(data, 30)
channel_data.to_csv('PW-Foundation.csv', index=False)

This code defines a function get_channel_video_details() that extracts the video details of the first n videos on the channel.

The function takes the ytInitialData dictionary as input and an integer n that specifies the number of videos to retrieve. The function calls the get_thumbnails(), get_time_of_posting(), get_title(), and get_videoUrl() functions to extract the thumbnail URLs, publication dates, titles, and video URLs of the first n videos on the channel. It then zips these lists together into a single list of tuples called main_data.

The function then creates a Pandas DataFrame from main_data and renames the columns to video_urls, title, thumbnail_url, and time_of_posting. Finally, the function returns the DataFrame.

The resulting DataFrame channel_data contains the video details of the first n videos on the channel, including the video URLs, titles, thumbnail URLs, and publication dates. The DataFrame is saved to a CSV file called PW-Foundation.csv with the to_csv() method.